Cristian Villazhannay  
ME 470  
May 21, 2023  

Implement a Backward Euler scheme to integrate in time, up to a final time of t = 0.5 for the 3D Heat Equation, using a Krylov Solver in PETSc to handle the parallel matrix solution.

$$
\frac{T^{n+1} - T^{n}}{\Delta t} = D\nabla^{2}T^{n+1}
$$

Please note that the linear matrix equation at every time step takes the form: 

$$
J(T^{n+1, k}) \delta T^{k} = -F(T^{n+1,k})
$$

### Part A 
For a given node, the discretized equation is: 
$$
\left(
1 + 2 \frac{D \Delta t}{\Delta x^2} + 2 \frac{D \Delta t}{\Delta y^2} + 2 \frac{D \Delta t}{\Delta z^2}
\right)
\delta^{k}_{i, j, k}

- \frac{D \Delta t}{\Delta x^2} \left( \delta^{k}_{i + 1,j    ,k    } + \delta^{k}_{i - 1,j    ,k    } \right)
- \frac{D \Delta t}{\Delta y^2} \left( \delta^{k}_{i    ,j + 1,k    } + \delta^{k}_{i    ,j - 1,k    } \right)
- \frac{D \Delta t}{\Delta z^2} \left( \delta^{k}_{i    ,j    ,k + 1} + \delta^{k}_{i    ,j    ,k - 1} \right)
= -F(u^{n+1}_{i, j, k})
$$

$$
\text{where }
$$

$$
-F(u^{n+1}_{i, j, k}) =
-u^{n+1,k}_{i,j,k} + u^{n,k}_{i,j,k}
+ \frac{D \Delta t}{\Delta x^2} \left( u_{i + 1, j    , k    }^{n+1} - 2 u_{i  , j  , k  }^{n+1} + u_{i - 1, j    , k    }^{n+1} \right) 
+ \frac{D \Delta t}{\Delta y^2} \left( u_{i    , j + 1, k    }^{n+1} - 2 u_{i  , j  , k  }^{n+1} + u_{i    , j - 1, k    }^{n+1} \right)
+ \frac{D \Delta t}{\Delta z^2} \left( u_{i    , j    , k + 1}^{n+1} - 2 u_{i  , j  , k  }^{n+1} + u_{i    , j    , k - 1}^{n+1} \right)
$$



### Part B 
The modified version of this equation so that the RHS resembles $J \cdot T^{n+1, k}$ and other terms is: 

$$
\left(
1 + 2 \frac{D \Delta t}{\Delta x^2} + 2 \frac{D \Delta t}{\Delta y^2} + 2 \frac{D \Delta t}{\Delta z^2}
\right)
\delta^{k}_{i, j, k}

- 2 \frac{D \Delta t}{\Delta x^2} \left( \delta^{k}_{i + 1,j    ,k    } + \delta^{k}_{i - 1,j    ,k    } \right)
- 2 \frac{D \Delta t}{\Delta y^2} \left( \delta^{k}_{i    ,j + 1,k    } + \delta^{k}_{i    ,j - 1,k    } \right)
- 2 \frac{D \Delta t}{\Delta z^2} \left( \delta^{k}_{i    ,j    ,k + 1} + \delta^{k}_{i    ,j    ,k - 1} \right)
= -F(u^{n+1}_{i, j, k})
$$

$$
\text{where }
$$

$$
-F(u^{n+1}_{i, j, k}) =
u^{n,k}_{i,j,k} 
+ u^{n+1,k}_{i,j,k} \left(-1 - 2\frac{D \Delta t}{\Delta x^2} - 2\frac{D \Delta t}{\Delta y^2} - 2 \frac{D \Delta t}{\Delta z^2} \right)
+ \frac{D \Delta t}{\Delta x^2} \left( u_{i + 1, j    , k    }^{n+1} + u_{i - 1, j    , k    }^{n+1} \right) 
+ \frac{D \Delta t}{\Delta y^2} \left( u_{i    , j + 1, k    }^{n+1} + u_{i    , j - 1, k    }^{n+1} \right)
+ \frac{D \Delta t}{\Delta z^2} \left( u_{i    , j    , k + 1}^{n+1} + u_{i    , j    , k - 1}^{n+1} \right)
$$


### Part C 
The final output of the case where $\Delta x=\Delta y =\Delta z = 0.01 \text{ and } \Delta t = 0.01$ is

`Hello`

### Part D 
Using the same grid size and timestep sizes as Part C, the value of T(0.5,0.5,0.5) is plotted as a function of time.